# Branching & The Merge Conflict

## The Git Paradigm: It's a Graph, Not a Timeline

This is the **most important concept** in Git. Most tutorials get this wrong.

### The Directed Acyclic Graph (DAG)
Git doesn't store differences (patches). It stores **snapshots**.

```
Commit C (HEAD)
    |
Commit B
    |
Commit A
```

But here's the magic: **Branches are just sticky notes**.

Visualize this:
```
      A --- B --- C  ← main (sticky note)
           \
            D --- E  ← feature (another sticky note)
```

- `A`, `B`, `C` are commit hashes (SHA-1 fingerprints of your files + metadata)
- `main` and `feature` are 41-byte files containing the hash of `C` and `E`
- **Creating a branch takes 0.1 seconds** because it's just writing a hash to a file

## Branching: Parallel Universes

**The Pointer Concept:**
A branch is a file containing a 40-character commit hash. Creating one takes milliseconds.

```bash
# Create and switch (modern syntax)
git switch -c feature/calculator-power
# Old syntax: git checkout -b feature/calculator-power

# See branches
git branch -v

# Switch between them
git switch main
git switch feature/calculator-power
```

### How Branching Works Internally

Before branch:
```
main: A --- B --- C
              ↑
            HEAD
```

After `git switch -c feature`:
```
main:    A --- B --- C
                   ↑
                 HEAD
                 feature
```

Both point to `C`. When you commit on `feature`:
```
main:    A --- B --- C
                   \
feature:             D
                      ↑
                    HEAD
```

The file `.git/refs/heads/feature` now contains the hash of `D`.

## Merging: Combining Histories

### Fast-Forward Merge
If `main` hasn't moved since you branched:
```
main:    A --- B --- C
                   ↑
feature:             D
```
Git simply moves `main` forward to `D`. History stays linear.

### 3-Way Merge (True Merge)
If `main` moved forward while you worked:
```
main:    A --- B --- C --- F (new commit)
                \
feature:          D --- E
```
Git creates a **merge commit** with two parents: `F` and `E`.

```bash
git switch main
git merge feature/calculator-power

# If no conflicts, editor opens for merge message
# Save and exit to complete
```

## Merge Conflicts (The Python Example)

**Scenario:**
You and a teammate both modify `calculator.py`.

Your version:
```python
def add(a, b):
    return a + b

def multiply(a, b):
    return a * b  # You added this
```

Teammate's version (in remote):
```python
def add(a, b):
    return a + b

def subtract(a, b):
    return a - b  # They added this
```

**The Conflict:**
When you `git pull` or `git merge`:
```
Auto-merging calculator.py
CONFLICT (content): Merge conflict in calculator.py
Automatic merge failed; fix conflicts and commit the result.
```

**The File Looks Like:**
```python
def add(a, b):
    return a + b

<<<<<<< HEAD
def multiply(a, b):
    return a * b
=======
def subtract(a, b):
    return a - b
>>>>>>> origin/main
```

**Resolution:**
1. Edit the file to keep both functions:
```python
def add(a, b):
    return a + b

def multiply(a, b):
    return a * b

def subtract(a, b):
    return a - b
```

2. Mark as resolved:
```bash
git add calculator.py
git commit -m "merge: Add both multiply and subtract functions"
```

**Golden Rule:** Don't just delete the `<<<<<<` markers—ensure the final code actually works and makes logical sense!

## Team Workflows Comparison

### GitFlow (The Classic)
**Best for:** Software with scheduled releases (e.g., version 1.0, 2.0)

**Branches:**
- `main`: Production only
- `develop`: Integration branch
- `feature/*`: New features (branch from develop)
- `release/*`: Prepare release (branch from develop, merge to main + develop)
- `hotfix/*`: Emergency fixes (branch from main, merge to main + develop)

**Workflow:**
1. Create `feature/payment` from `develop`
2. Work, commit, push
3. Merge to `develop` via Pull Request
4. When ready: branch `release/v1.0` from `develop`
5. Bug fixes go to `release/v1.0`
6. Merge `release/v1.0` to `main` (tag v1.0) and back to `develop`

### GitHub Flow (The Simple)
**Best for:** Web applications, continuous deployment

**Branches:**
- `main`: Always deployable
- `feature/*`: Short-lived branches

**Workflow:**
1. Create `feature/add-auth` from `main`
2. Work, commit, push
3. Open Pull Request to `main`
4. Review, merge, deploy immediately

**Pros:** Simple, fast. **Cons:** No staging environment for big releases.

### Trunk-Based Development (The Extreme)
**Best for:** Large teams, CI/CD experts

**Concept:** Everyone commits to `main` (trunk) frequently (multiple times/day). No long-lived branches.

**Technique:**
- **Branch by Abstraction:** Keep unfinished features behind feature flags
- **Short-lived branches:** Max 1-day lifetime
- **Pair programming:** Reduces need for code review delays

**Comparison Table:**

| Aspect | GitFlow | GitHub Flow | Trunk-Based |
|--------|---------|-------------|-------------|
| Branches | Many long-lived | Few short-lived | Almost none |
| Releases | Scheduled | Continuous | Continuous |
| Complexity | High | Medium | Low (but hard discipline) |
| Team Size | Large enterprise | Small-medium | Large (with automation) |

## Fork & Pull Request (Open Source Workflow)

**When:** Contributing to projects where you don't have write access.

```bash
# 1. Fork on GitHub (creates yourname/original-repo)
# 2. Clone YOUR fork
git clone git@github.com:yourname/repo.git

# 3. Add upstream remote (the original)
git remote add upstream git@github.com:original/repo.git

# 4. Keep your main updated
git switch main
git fetch upstream
git rebase upstream/main  # Or: git merge upstream/main
git push origin main

# 5. Create feature branch
git switch -c feature/fix-typo

# 6. Work, commit, push to your fork
git push origin feature/fix-typo

# 7. Open Pull Request on GitHub (original repo ← your fork)
```